In [38]:
import os
import sys
os.chdir("E:\\2_Studium\\1_Mannheim\\4_Semester\\Masterarbeit\\code\\master-thesis-code\\data")
import pandas as pd

In [39]:
feature_abt_buy_test = pd.read_csv("abt_buy/features_abt_buy_test")
feature_abt_buy_train = pd.read_csv("abt_buy/features_abt_buy_train")
abt_buy_gold_standard_train = pd.read_csv("abt_buy/abt_buy_gold_standard_train.csv",sep=";")
abt_buy_gold_standard_test = pd.read_csv("abt_buy/abt_buy_gold_standard_test.csv",sep=";")

feature_amazon_google_test = pd.read_csv("amazon_google/features_amazon_google_test")
feature_amazon_google_train = pd.read_csv("amazon_google/features_amazon_google_train")
gs_amazon_google_train = pd.read_csv("amazon_google/amazon_google_gold_standard_train.csv",sep=";")
gs_amazon_google_test = pd.read_csv("amazon_google/amazon_google_gold_standard_test.csv",sep=";")

feature_wdc_phones_test = pd.read_csv("wdc_product/features_phones_phones_catalog_test")
feature_wdc_phones_train = pd.read_csv("wdc_product/features_phones_phones_catalog_train")
gs_wdc_phones_train = pd.read_csv("wdc_product/phones_phones_catalog_gold_standard_train.csv",sep=";")
gs_wdc_phones_test = pd.read_csv("wdc_product/phones_phones_catalog_gold_standard_test.csv",sep=";")


feature_wdc_headphones_test = pd.read_csv("wdc_product/features_headphones_headphones_catalog_test")
feature_wdc_headphones_train = pd.read_csv("wdc_product/features_headphones_headphones_catalog_train")
gs_wdc_headphones_train = pd.read_csv("wdc_product/headphones_headphones_catalog_silver_standard_train.csv",sep=";")
gs_wdc_headphones_test = pd.read_csv("wdc_product/headphones_headphones_catalog_silver_standard_test.csv",sep=";")

feature_dbpedia_viaf_test = pd.read_csv("author/features_DBPediaAuthors_VIAFDataAuthors_test")
feature_dbpedia_viaf_train = pd.read_csv("author/features_DBPediaAuthors_VIAFDataAuthors_train")
gs_dbpedia_viaf_test = pd.read_csv("author/DBPediaAuthors_VIAFDataAuthors_gold_standard_test.csv",sep=";")
gs_dbpedia_viaf_train = pd.read_csv("author/DBPediaAuthors_VIAFDataAuthors_gold_standard_train.csv",sep=";")

feature_dbpedia_dnb_test = pd.read_csv("author/features_DBPediaAuthors_DnbDataAuthors_test")
feature_dbpedia_dnb_train = pd.read_csv("author/features_DBPediaAuthors_DnbDataAuthors_train")
gs_dbpedia_dnb_test = pd.read_csv("author/DBPediaAuthors_DnbDataAuthors_gold_standard_test.csv",sep=";")
gs_dbpedia_dnb_train = pd.read_csv("author/DBPediaAuthors_DnbDataAuthors_gold_standard_train.csv",sep=";")

#feature_dbpedia_wiki_test = pd.read_csv("author/features_DBPediaAuthors_WikiDataAuthors_test")
#feature_dbpedia_wiki_train = pd.read_csv("author/features_DBPediaAuthors_WikiDataAuthors_train")

In [40]:
# Data profiling information
metadata = [
    ['abt_buy',feature_abt_buy_train.shape[1],
    feature_abt_buy_train.shape[0],feature_abt_buy_test.shape[0]],
    ['amazon_google',feature_amazon_google_train.shape[1],
    feature_amazon_google_train.shape[0],feature_amazon_google_test.shape[0]],
    ['wdc_phones',feature_wdc_phones_train.shape[1],
    feature_wdc_phones_train.shape[0],feature_wdc_phones_test.shape[0]],
    ['wdc_headphones',feature_wdc_headphones_train.shape[1],
    feature_wdc_headphones_train.shape[0],feature_wdc_headphones_test.shape[0]],
    ['dbpedia_viaf',feature_dbpedia_viaf_train.shape[1],
    feature_dbpedia_viaf_train.shape[0],feature_dbpedia_viaf_test.shape[0]],
    #['dbpedia_wiki',feature_dbpedia_wiki_train.shape[1],
    #feature_dbpedia_wiki_train.shape[0],feature_dbpedia_wiki_test.shape[0]],
    ['dbpedia_dnb',feature_dbpedia_dnb_train.shape[1],
    feature_dbpedia_dnb_train.shape[0],feature_dbpedia_dnb_test.shape[0]],
    ]
df_profiling = pd.DataFrame(metadata, columns=['Dataset', '# Attributes', '# Train', '# Test'])
df_profiling

,Dataset,# Attributes,# Train,# Test
0,abt_buy,19,5730,1432
1,amazon_google,24,6753,1687
2,wdc_phones,135,1762,440
3,wdc_headphones,142,1163,290
4,dbpedia_viaf,27,15316,3828
5,dbpedia_dnb,26,13863,3465


# Preprocessing abt_buy

In [3]:
ids_abt_buy_train = feature_abt_buy_train[["source_id","target_id"]]
ids_abt_buy_test = feature_abt_buy_test[["source_id","target_id"]]
abt_original = pd.read_csv("abt_buy/abt.csv",encoding='latin1').add_prefix('left_')
buy_original = pd.read_csv("abt_buy/buy.csv",encoding='latin1').add_prefix('right_')

#train
abt_buy_train = ids_abt_buy_train.merge(
    abt_original,left_on="source_id",right_on="left_subject_id").merge(
    buy_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
abt_buy_train['id'] = abt_buy_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#test
abt_buy_test = ids_abt_buy_test.merge(
    abt_original,left_on="source_id",right_on="left_subject_id").merge(
    buy_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
abt_buy_test['id'] = abt_buy_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

In [39]:
#train
abt_buy_gold_standard_train['id'] = abt_buy_gold_standard_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
abt_buy_gold_standard_train.drop(['source_id','target_id'],axis=1,inplace=True)

#test
abt_buy_gold_standard_test['id'] = abt_buy_gold_standard_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
abt_buy_gold_standard_test.drop(['source_id','target_id'],axis=1,inplace=True)

In [40]:
abt_buy_train = abt_buy_train.merge(abt_buy_gold_standard_train,on='id')
abt_buy_train = abt_buy_train.rename(columns={'matching':'label'})
abt_buy_train.drop('right_manufacturer',axis=1,inplace=True)
abt_buy_train['label'] = abt_buy_train['label'].astype(int)

In [41]:
abt_buy_test = abt_buy_test.merge(abt_buy_gold_standard_test,on='id')
abt_buy_test = abt_buy_test.rename(columns={'matching':'label'})
abt_buy_test.drop('right_manufacturer',axis=1,inplace=True)
abt_buy_test['label'] = abt_buy_test['label'].astype(int)
abt_buy_test.head()

,source_id,target_id,left_name,left_description,left_price,right_name,right_description,right_price,id,label
0,17187,202041204,Universal IR/RF Remote - MX350,Universal IR/RF Remote - MX350/ Controls Up To...,149.95,Universal MX-350 Osiris Remote Control - MX350,"TV, VCR, DVD Player, CD Player, Cable Box, Sat...",93.9,17187-202041204,1
1,17187,203069939,Universal IR/RF Remote - MX350,Universal IR/RF Remote - MX350/ Controls Up To...,149.95,Universal MasterControl Universal Learning Rem...,"Audio/Video, TV, VCR, DVD Player, CD Player, C...",NaN,17187-203069939,0
2,33965,203069939,Logitech Harmony One Advanced Universal Remote...,Logitech Harmony One Advanced Universal Remote...,249.00,Universal MasterControl Universal Learning Rem...,"Audio/Video, TV, VCR, DVD Player, CD Player, C...",NaN,33965-203069939,0
3,31273,203069939,Sony Universal Remote Commander Remote Control...,Sony Universal Remote Commander Remote Control...,NaN,Universal MasterControl Universal Learning Rem...,"Audio/Video, TV, VCR, DVD Player, CD Player, C...",NaN,31273-203069939,0
4,27779,205844283,Sony LCS-CSQ/B Black Soft Carrying Case - LCSCSQB,Sony LCS-CSQ/B Black Soft Carrying Case - LCSC...,NaN,Sony LCS-CSQ Soft Cyber-shot Camera Case - LCS...,Top Loading - Polyamide - Black,NaN,27779-205844283,1


## Split Train into Train and Validation

In [77]:
y = abt_buy_train['label']

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(abt_buy_train, y,
                                                stratify=y, 
                                                test_size=0.25)

NameError: name 'abt_buy_train' is not defined

In [80]:
abt_buy_train = X_train
abt_buy_validation = X_val

In [81]:
abt_buy_train.to_csv("abt_buy/abt_buy_train",index=False)
abt_buy_validation.to_csv("abt_buy/abt_buy_validation",index=False)
abt_buy_test.to_csv("abt_buy/abt_buy_test",index=False)

# Preprocessing dbpedia_viaf

In [69]:
ids_dbpedia_viaf_train = feature_dbpedia_viaf_train[["source_id","target_id"]]
ids_dbpedia_viaf_test = feature_dbpedia_viaf_test[["source_id","target_id"]]
dbpedia_original = pd.read_csv("author/DBpediaAuthors.csv",encoding='latin1').add_prefix('left_')
viaf_original = pd.read_csv("author/VIAFDataAuthors.csv",encoding='latin1').add_prefix('right_')

#train
dbpedia_viaf_train = ids_dbpedia_viaf_train.merge(
    dbpedia_original,left_on="source_id",right_on="left_subject_id").merge(
    viaf_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
dbpedia_viaf_train['id'] = dbpedia_viaf_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#test
dbpedia_viaf_test = ids_dbpedia_viaf_test.merge(
    dbpedia_original,left_on="source_id",right_on="left_subject_id").merge(
    viaf_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
dbpedia_viaf_test['id'] = dbpedia_viaf_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

In [70]:
#train
gs_dbpedia_viaf_train['id'] = gs_dbpedia_viaf_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
gs_dbpedia_viaf_train.drop(['source_id','target_id'],axis=1,inplace=True)

#test
gs_dbpedia_viaf_test['id'] = gs_dbpedia_viaf_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
gs_dbpedia_viaf_test.drop(['source_id','target_id'],axis=1,inplace=True)

In [71]:
dbpedia_viaf_train = dbpedia_viaf_train.merge(gs_dbpedia_viaf_train,on='id')
dbpedia_viaf_train = dbpedia_viaf_train.rename(columns={'matching':'label'})
dbpedia_viaf_train.drop(['left_abstract','left_links'],axis=1,inplace=True)
dbpedia_viaf_train['label'] = dbpedia_viaf_train['label'].astype(int)

In [73]:
dbpedia_viaf_test = dbpedia_viaf_test.merge(gs_dbpedia_viaf_test,on='id')
dbpedia_viaf_test = dbpedia_viaf_test.rename(columns={'matching':'label'})
dbpedia_viaf_test.drop(['left_abstract','left_links'],axis=1,inplace=True)
dbpedia_viaf_test['label'] = dbpedia_viaf_test['label'].astype(int)

## Split Train into Train and Validation


In [76]:
y = dbpedia_viaf_train['label']
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(dbpedia_viaf_train, y,
                                                stratify=y, 
                                                test_size=0.25)
dbpedia_viaf_train = X_train
dbpedia_viaf_validation = X_val
dbpedia_viaf_train.to_csv("author/dbpedia_viaf_train",index=False)
dbpedia_viaf_validation.to_csv("author/dbpedia_viaf_validation",index=False)
dbpedia_viaf_test.to_csv("author/dbpedia_viaf_test",index=False)

In [95]:
# Data profiling information blocked pairs
metadata_2 = [['dbpedia_viaf',dbpedia_viaf_train.shape[0],dbpedia_viaf_validation.shape[0],dbpedia_viaf_test.shape[0]]]
df_blocking = pd.DataFrame(metadata_2, columns=['Dataset', '# Train', '# Validation', '# Test'])
df_blocking

,Dataset,# Train,# Validation,# Test
0,dbpedia_viaf,10885,3629,3651


In [101]:
print(sum(dbpedia_viaf_train['label'] == 1))
print(sum(dbpedia_viaf_train['label'] == 0))
print(sum(dbpedia_viaf_validation['label'] == 1))
print(sum(dbpedia_viaf_validation['label'] == 0))
print(sum(dbpedia_viaf_test['label'] == 1))
print(sum(dbpedia_viaf_test['label'] == 0))

1759
9126
587
3042
591
3060


# Preprocessing dbpedia_dnb

In [38]:
from sklearn.model_selection import train_test_split

ids_dbpedia_dnb_train = feature_dbpedia_dnb_train[["source_id","target_id"]]
ids_dbpedia_dnb_test = feature_dbpedia_dnb_test[["source_id","target_id"]]
dbpedia_original = pd.read_csv("author/DBpediaAuthors.csv",encoding='latin1').add_prefix('left_')
dnb_original = pd.read_csv("author/DnbDataAuthors.csv",encoding='latin1').add_prefix('right_')

#train
dbpedia_dnb_train = ids_dbpedia_dnb_train.merge(
    dbpedia_original,left_on="source_id",right_on="left_subject_id").merge(
    dnb_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
dbpedia_dnb_train['id'] = dbpedia_dnb_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#test
dbpedia_dnb_test = ids_dbpedia_dnb_test.merge(
    dbpedia_original,left_on="source_id",right_on="left_subject_id").merge(
    dnb_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
dbpedia_dnb_test['id'] = dbpedia_dnb_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#train
gs_dbpedia_dnb_train['id'] = gs_dbpedia_dnb_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
gs_dbpedia_dnb_train.drop(['source_id','target_id'],axis=1,inplace=True)

#test
gs_dbpedia_dnb_test['id'] = gs_dbpedia_dnb_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
gs_dbpedia_dnb_test.drop(['source_id','target_id'],axis=1,inplace=True)

dbpedia_dnb_train = dbpedia_dnb_train.merge(gs_dbpedia_dnb_train,on='id')
dbpedia_dnb_train = dbpedia_dnb_train.rename(columns={'matching':'label'})
dbpedia_dnb_train.drop(
    ['left_abstract','left_links','left_work',
    'right_forename','right_nameAddition','right_personalName',
    'right_prefix',
    'right_surname',
    'right_work'],axis=1,inplace=True)
dbpedia_dnb_train['label'] = dbpedia_dnb_train['label'].astype(int)

dbpedia_dnb_test = dbpedia_dnb_test.merge(gs_dbpedia_dnb_test,on='id')
dbpedia_dnb_test = dbpedia_dnb_test.rename(columns={'matching':'label'})
dbpedia_dnb_test.drop(
    ['left_abstract','left_links','left_work',
    'right_forename','right_nameAddition','right_personalName',
    'right_prefix',
    'right_surname',
    'right_work'],axis=1,inplace=True)
dbpedia_dnb_test['label'] = dbpedia_dnb_test['label'].astype(int)

y = dbpedia_dnb_train['label']
X_train, X_val, y_train, y_val = train_test_split(dbpedia_dnb_train, y, stratify=y, test_size=0.25)

# Split Train into Train and Validation
dbpedia_dnb_train = X_train
dbpedia_dnb_validation = X_val
dbpedia_dnb_train.to_csv("author/dbpedia_dnb_train",index=False)
dbpedia_dnb_validation.to_csv("author/dbpedia_dnb_validation",index=False)
dbpedia_dnb_test.to_csv("author/dbpedia_dnb_test",index=False)

In [42]:
print(sum(dbpedia_dnb_train['label'] == 1))
print(sum(dbpedia_dnb_train['label'] == 0))
print(sum(dbpedia_dnb_validation['label'] == 1))
print(sum(dbpedia_dnb_validation['label'] == 0))
print(sum(dbpedia_dnb_test['label'] == 1))
print(sum(dbpedia_dnb_test['label'] == 0))

1401
7418
467
2473
469
2493


# Preprocessing amazon_google

In [10]:
from sklearn.model_selection import train_test_split

ids_amazon_google_train = feature_amazon_google_train[["source_id","target_id"]]
ids_amazon_google_test = feature_amazon_google_test[["source_id","target_id"]]
amazon_original = pd.read_csv("amazon_google/amazon.csv",encoding='latin1').add_prefix('left_')
google_original = pd.read_csv("amazon_google/google.csv",encoding='latin1').add_prefix('right_')

#train
amazon_google_train = ids_amazon_google_train.merge(
    amazon_original,left_on="source_id",right_on="left_subject_id").merge(
    google_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
amazon_google_train['id'] = amazon_google_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#test
amazon_google_test = ids_amazon_google_test.merge(
    amazon_original,left_on="source_id",right_on="left_subject_id").merge(
    google_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
amazon_google_test['id'] = amazon_google_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#train
gs_amazon_google_train['id'] = gs_amazon_google_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
gs_amazon_google_train.drop(['source_id','target_id'],axis=1,inplace=True)

#test
gs_amazon_google_test['id'] = gs_amazon_google_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
gs_amazon_google_test.drop(['source_id','target_id'],axis=1,inplace=True)

amazon_google_train = amazon_google_train.merge(gs_amazon_google_train,on='id')
amazon_google_train = amazon_google_train.rename(columns={'matching':'label'})
amazon_google_train['label'] = amazon_google_train['label'].astype(int)

amazon_google_test = amazon_google_test.merge(gs_amazon_google_test,on='id')
amazon_google_test = amazon_google_test.rename(columns={'matching':'label'})
amazon_google_test['label'] = amazon_google_test['label'].astype(int)

y = amazon_google_train['label']
X_train, X_val, y_train, y_val = train_test_split(amazon_google_train, y, stratify=y, test_size=0.25)

# Split Train into Train and Validation
amazon_google_train = X_train
amazon_google_validation = X_val
amazon_google_train.to_csv("amazon_google/amazon_google_train",index=False)
amazon_google_validation.to_csv("amazon_google/amazon_google_validation",index=False)
amazon_google_test.to_csv("amazon_google/amazon_google_test",index=False)

In [11]:
print(sum(amazon_google_train['label'] == 1))
print(sum(amazon_google_train['label'] == 0))
print(sum(amazon_google_validation['label'] == 1))
print(sum(amazon_google_validation['label'] == 0))
print(sum(amazon_google_test['label'] == 1))
print(sum(amazon_google_test['label'] == 0))

779
4285
260
1429
259
1428


# Preprocessing wdc_phones

In [22]:
from sklearn.model_selection import train_test_split

ids_wdc_phones_train = feature_wdc_phones_train[["source_id","target_id"]]
ids_wdc_phones_test = feature_wdc_phones_test[["source_id","target_id"]]
phones_original = pd.read_csv("wdc_product/phones.csv",sep='\|\|').drop('Unnamed: 29',axis=1).add_prefix('left_')
phones_catalog_original = pd.read_csv("wdc_product/phones_catalog.csv",sep='\|\|').drop('Unnamed: 31',axis=1).add_prefix('right_')

#train
wdc_phones_train = ids_wdc_phones_train.merge(
    phones_original,left_on="source_id",right_on="left_subject_id").merge(
    phones_catalog_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
wdc_phones_train['id'] = wdc_phones_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#test
wdc_phones_test = ids_wdc_phones_test.merge(
    phones_original,left_on="source_id",right_on="left_subject_id").merge(
    phones_catalog_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
wdc_phones_test['id'] = wdc_phones_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#train
gs_wdc_phones_train['id'] = gs_wdc_phones_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
gs_wdc_phones_train.drop(['source_id','target_id'],axis=1,inplace=True)

#test
gs_wdc_phones_test['id'] = gs_wdc_phones_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
gs_wdc_phones_test.drop(['source_id','target_id'],axis=1,inplace=True)

wdc_phones_train = wdc_phones_train.merge(gs_wdc_phones_train,on='id')
wdc_phones_train = wdc_phones_train.rename(columns={'matching':'label'})
wdc_phones_train.drop(
    ['left_<http://schema.org/Product/warc>',
    'left_<http://schema.org/Product/url>',
    'right_<http://schema.org/Product/voltage>',
    'right_<http://schema.org/Product/package_height>',
    'right_<http://schema.org/Product/core_count>',
    'right_<http://schema.org/Product/power_supply>'],axis=1,inplace=True)
wdc_phones_train['label'] = wdc_phones_train['label'].astype(int)

wdc_phones_test = wdc_phones_test.merge(gs_wdc_phones_test,on='id')
wdc_phones_test = wdc_phones_test.rename(columns={'matching':'label'})
wdc_phones_test.drop(
    ['left_<http://schema.org/Product/warc>',
    'left_<http://schema.org/Product/url>',
    'right_<http://schema.org/Product/voltage>',
    'right_<http://schema.org/Product/package_height>',
    'right_<http://schema.org/Product/core_count>',
    'right_<http://schema.org/Product/power_supply>'],axis=1,inplace=True)
wdc_phones_test['label'] = wdc_phones_test['label'].astype(int)

y = wdc_phones_train['label']
X_train, X_val, y_train, y_val = train_test_split(wdc_phones_train, y, stratify=y, test_size=0.25)

# Split Train into Train and Validation
wdc_phones_train = X_train
wdc_phones_validation = X_val
wdc_phones_train.to_csv("wdc_product/wdc_phones_train",index=False)
wdc_phones_validation.to_csv("wdc_product/wdc_phones_validation",index=False)
wdc_phones_test.to_csv("wdc_product/wdc_phones_test",index=False)

ipykernel_launcher:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
ipykernel_launcher:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [23]:
print(sum(wdc_phones_train['label'] == 1))
print(sum(wdc_phones_train['label'] == 0))
print(sum(wdc_phones_validation['label'] == 1))
print(sum(wdc_phones_validation['label'] == 0))
print(sum(wdc_phones_test['label'] == 1))
print(sum(wdc_phones_test['label'] == 0))

154
1167
52
389
51
389


# Preprocessing wdc_headphones

In [41]:
from sklearn.model_selection import train_test_split

ids_wdc_headphones_train = feature_wdc_headphones_train[["source_id","target_id"]]
ids_wdc_headphones_test = feature_wdc_headphones_test[["source_id","target_id"]]
headphones_original = pd.read_csv("wdc_product/headphones.csv",sep='\|\|').drop('Unnamed: 30',axis=1).add_prefix('left_')
headphones_catalog_original = pd.read_csv("wdc_product/headphones_catalog.csv",sep='\|\|').drop('Unnamed: 39',axis=1).add_prefix('right_')

#train
wdc_headphones_train = ids_wdc_headphones_train.merge(
    headphones_original,left_on="source_id",right_on="left_subject_id").merge(
    headphones_catalog_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
wdc_headphones_train['id'] = wdc_headphones_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#test
wdc_headphones_test = ids_wdc_headphones_test.merge(
    headphones_original,left_on="source_id",right_on="left_subject_id").merge(
    headphones_catalog_original,left_on="target_id",right_on="right_subject_id").drop(['left_subject_id','right_subject_id'],1)
wdc_headphones_test['id'] = wdc_headphones_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)

#train
gs_wdc_headphones_train['id'] = gs_wdc_headphones_train[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
gs_wdc_headphones_train.drop(['source_id','target_id'],axis=1,inplace=True)

#test
gs_wdc_headphones_test['id'] = gs_wdc_headphones_test[['source_id','target_id']].astype(str).agg('-'.join,axis=1)
gs_wdc_headphones_test.drop(['source_id','target_id'],axis=1,inplace=True)

wdc_headphones_train = wdc_headphones_train.merge(gs_wdc_headphones_train,on='id')
wdc_headphones_train = wdc_headphones_train.rename(columns={'matching':'label'})
wdc_headphones_train.drop(
    ['left_<http://schema.org/Product/warc>',
    'left_<http://schema.org/Product/url>',
    'right_<http://schema.org/Product/mdoel>',
    'right_<http://schema.org/Product/aditional_features>',
    'right_<http://schema.org/Product/depth>',
    'right_<http://schema.org/Product/sound_output_mode>',
    'right_<http://schema.org/Product/detachable_cable>',
    'right_<http://schema.org/Product/foldable>',
    'right_<http://schema.org/Product/controls>',
    'right_<http://schema.org/Product/microphone_response>',
    'right_<http://schema.org/Product/microphone_audio_details>',
    'right_<http://schema.org/Product/diaphragm>',
    'right_<http://schema.org/Product/compliant_standards>'],axis=1,inplace=True)
wdc_headphones_train['label'] = wdc_headphones_train['label'].astype(int)

wdc_headphones_test = wdc_headphones_test.merge(gs_wdc_headphones_test,on='id')
wdc_headphones_test = wdc_headphones_test.rename(columns={'matching':'label'})
wdc_headphones_test.drop(
    ['left_<http://schema.org/Product/warc>',
    'left_<http://schema.org/Product/url>',
    'right_<http://schema.org/Product/mdoel>',
    'right_<http://schema.org/Product/aditional_features>',
    'right_<http://schema.org/Product/depth>',
    'right_<http://schema.org/Product/sound_output_mode>',
    'right_<http://schema.org/Product/detachable_cable>',
    'right_<http://schema.org/Product/foldable>',
    'right_<http://schema.org/Product/controls>',
    'right_<http://schema.org/Product/microphone_response>',
    'right_<http://schema.org/Product/microphone_audio_details>',
    'right_<http://schema.org/Product/diaphragm>',
    'right_<http://schema.org/Product/compliant_standards>'],axis=1,inplace=True)
wdc_headphones_test['label'] = wdc_headphones_test['label'].astype(int)

y = wdc_headphones_train['label']
X_train, X_val, y_train, y_val = train_test_split(wdc_headphones_train, y, stratify=y, test_size=0.25)

# Split Train into Train and Validation
wdc_headphones_train = X_train
wdc_headphones_validation = X_val
wdc_headphones_train.to_csv("wdc_product/wdc_headphones_train",index=False)
wdc_headphones_validation.to_csv("wdc_product/wdc_headphones_validation",index=False)
wdc_headphones_test.to_csv("wdc_product/wdc_headphones_test",index=False)

ipykernel_launcher:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
ipykernel_launcher:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [42]:
print(sum(wdc_headphones_train['label'] == 1))
print(sum(wdc_headphones_train['label'] == 0))
print(sum(wdc_headphones_validation['label'] == 1))
print(sum(wdc_headphones_validation['label'] == 0))
print(sum(wdc_headphones_test['label'] == 1))
print(sum(wdc_headphones_test['label'] == 0))

135
737
45
246
45
245


In [32]:
pd.read_csv("wdc_product/headphones_catalog.csv",sep='\|\|').columns

ipykernel_launcher:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


Index(['subject_id', '<http://schema.org/Product/product_gtin>',
       '<http://schema.org/Product/cables_included>',
       '<http://schema.org/Product/weight>',
       '<http://schema.org/Product/mdoel>',
       '<http://schema.org/Product/aditional_features>',
       '<http://schema.org/Product/sensitivity>',
       '<http://schema.org/Product/series>',
       '<http://schema.org/Product/model>',
       '<http://schema.org/Product/headphones_form_factor>',
       '<http://schema.org/Product/width>',
       '<http://schema.org/Product/depth>',
       '<http://schema.org/Product/sound_output_mode>',
       '<http://schema.org/Product/thd>',
       '<http://schema.org/Product/detachable_cable>',
       '<http://schema.org/Product/brand>',
       '<http://schema.org/Product/foldable>',
       '<http://schema.org/Product/microphone_sensitivity>',
       '<http://schema.org/Product/max_input_power>',
       '<http://schema.org/Product/product_type>',
       '<http://schema.org/Product/he